In [59]:
import numpy as np
import pandas as pd
import sys
from PEIOPT_for_loop import *
import csv
import time

x_all = pd.read_csv("./../x_all.csv", header=None)
y_all = pd.read_csv("./../y_all.csv", header=None)
cas_data = pd.read_csv("./../sum_log.csv")
counter_i = 1

csv_title_list = ["trial","cas_number","pred_value","real_value","error_value"]

f = open('PEIOPT_process_10_1.csv', 'w')
writer = csv.writer(f, lineterminator='\n')
writer.writerow(csv_title_list)

sampler = np.random.permutation(len(x_all))

learn_x = x_all.take(sampler[:10])
learn_y = y_all.take(sampler[:10])
cas_source = (cas_data.ix[:,1]).take(sampler[10:])
cas_source = list(cas_source.values.flatten())
#
test_x = x_all.take(sampler[10:])
test_y = y_all.take(sampler[10:])
#

#こっからループ
for i in range(20):
    learn_x.to_csv("./x_10_1.csv", index=False, header=False)
    learn_y.to_csv("./y_10_1.csv", index=False, header=False)
    test_x.to_csv("./xeval_10_1.csv", index=False, header=False)
    test_y.to_csv("./yeval_10_1.csv", index=False, header=False)


    #最適化手法本体

    # データロード
    X = np.loadtxt("./x_10_1.csv",delimiter=",")
    X = X.astype(np.float64)#設計変数
    ycsv = pd.read_csv("./y_10_1.csv", header=None)
    add_rand = np.random.rand(len(ycsv)) * np.random.rand(len(ycsv)) *0.1
    for i in range(len(ycsv)):
        ycsv.ix[i,0] = ycsv.ix[i,0] + add_rand[i]
    y = ycsv.as_matrix()
    y = y.astype(np.float64)#特性
    boundcsv = np.loadtxt("bound.csv",delimiter=",")#設計変数のレンジ
    yinfcsv = np.loadtxt("yinf.csv",delimiter=",")#各特性に対する要求
    option = pd.read_csv("option.csv", header=None)#option

    #二次元配列に統一
    if boundcsv.ndim == 1:
        boundcsv = boundcsv[:,np.newaxis]
        X = X[:,np.newaxis]
    if yinfcsv.ndim == 1:
        yinfcsv = yinfcsv[:,np.newaxis]

    #boundとyinfをリストの形にする
    bounds = []
    yinf = []
    for var in range(0, boundcsv.shape[1]):
        bounds.append([boundcsv[0,var],boundcsv[1,var]])
    for var in range(0, yinfcsv.shape[1]):
        yinf.append([yinfcsv[0,var],yinfcsv[1,var],yinfcsv[2,var]])

    optfunc_first = []
    Xselec = []
    theta = []
    result = []
    # 任意の数の候補を選択するまでループ
    for i in range(0,int(option[1][1])):
        # GPモデルの構築
        if i == 0: #最初は全ての特性値に対してモデルを構築する
            gp_list = []#各特性ごとに作成したGPモデルを格納するリスト
            for obj in range(0,len(yinf)):
                gp = GaussianProcess(thetaL = 0.0000001, thetaU = 100)
                Xdb = X[np.isnan(y[:,obj]) == False, :]#yが欠損値の場合は無視
                ydb = y[np.isnan(y[:,obj]) == False, obj]#yが欠損値の場合は無視
                gp.fit(Xdb, ydb)
                gp_list.append(gp)

        else:#二回目以降は最適化する特性のみのモデルを再構築する。
            gp = GaussianProcess(theta0 = theta)
            Xdb = X[np.isnan(y[:,optfunc_first.optobj]) == False, :]#yが欠損値の場合は無視
            ydb = y[np.isnan(y[:,optfunc_first.optobj]) == False, optfunc_first.optobj]#yが欠損値の場合は無視

            #ダミーデータをデータベースに追加
            Xdb = np.r_[Xdb, Xselec]
            ydb = np.r_[ydb[:,np.newaxis], optfunc_first.CL[int(option[1][2])]*np.ones([i,1])]

            gp.fit(Xdb, ydb)
            gp_list[optfunc_first.optobj] = gp

        # DEの準備
        optfunc = Opt_func(y = y, model = gp_list, yinfdata = yinf, option = option)#最適化する関数を含むオブジェクトを作成  
        if i == 0:
            #最初に作成したモデルは保存しておく
            gp_list_first = gp_list.copy()
            optfunc_first = Opt_func(y = y, model = gp_list_first, yinfdata = yinf, option = option)
            #2回目以降はこのハイパーパラメータを用いてモデルを構築する
            theta = gp_list[optfunc_first.optobj].theta_

        # DE
        if int(option[1][4]) == 0:
            if int(option[1][0]) == 0:
                de_result = differential_evolution(optfunc.P, bounds)#Pで選択
            elif int(option[1][0]) == 1:
                de_result = differential_evolution(optfunc.EI, bounds)#EIで選択
            elif int(option[1][0]) == 2:
                de_result = differential_evolution(optfunc.Y, bounds)#予測値で選択

        elif int(option[1][4]) == 1:
            #評価を行うリストを読み込み
            Xeval = np.loadtxt("./xeval_10_1.csv",delimiter=",")
            Xeval = Xeval.astype(np.float64)

            #評価
            if int(option[1][0]) == 0:
                eval = optfunc.P(Xeval)#Pで選択
            elif int(option[1][0]) == 1:
                eval = optfunc.EI(Xeval)#EIで選択
            elif int(option[1][0]) == 2:
                eval = optfunc.Y(Xeval)#予測値で選択

            #候補の選択
            de_result = result_cont(Xeval[eval.argmin(),:],eval.argmin())

        #選択された候補を格納
        if i == 0:
            Xselec = de_result.x[:,np.newaxis].T
        else:
            Xselec = np.r_[Xselec, de_result.x[:,np.newaxis].T]

        #出力用のデータフレームを作成
        result_temp = pd.DataFrame(de_result.x.T).T#Xを格納
        result_temp[result_temp.shape[1]] = de_result.message#DEの最適化の終了メッセージを格納

        #最初に作成したモデル（ダミーデータを含まない）で評価を行い、その評価値を格納する
        if int(option[1][0]) == 0:
            eve = optfunc_first.P(de_result.x[:,np.newaxis].T)
        elif int(option[1][0]) == 1:
            eve = optfunc_first.EI(de_result.x[:,np.newaxis].T)
            result_of_y_pred = optfunc_first.ypred
        elif int(option[1][0]) == 2:
            eve = optfunc_first.Y(de_result.x[:,np.newaxis].T)
            if optfunc_first.optdirection < 0:
                eve = -eve

        result_temp[result_temp.shape[1]] = -eve
        result_temp[result_temp.shape[1]] = "id"
        result_temp[result_temp.shape[1]] = '-' if (int(option[1][4]) == 0) else de_result.id


        #出力用のデータフレームを更新
        if i == 0:
            result = result_temp
        else:
            result = pd.concat([result,result_temp], axis=0)
    y_pred_value = pd.Series(result_of_y_pred[0])
    print(y_pred_value)
    result = pd.concat([result,y_pred_value], axis=1)
    #print("succeed")
    print(cas_source[de_result.id])

    y_eval = pd.read_csv("./yeval_10_1.csv", header=None)
    y_real_value = y_eval.ix[de_result.id,0]

    write_list = []
    write_list.append(counter_i)
    counter_i += 1
    write_list.append(cas_source[de_result.id])
    write_list.append(result_of_y_pred[0][0])
    write_list.append(y_eval.ix[de_result.id,0])
    print(y_eval.ix[de_result.id,0])
    error_value = abs(result_of_y_pred[0][0] - y_eval.ix[de_result.id,0])
    write_list.append(error_value)
    writer.writerow(write_list)
    print(de_result.id)

    learn_x = learn_x.append(test_x.iloc[de_result.id,:])
    learn_y = learn_y.append(test_y.iloc[de_result.id,:])
    test_x = test_x.drop(de_result.id)
    test_y = test_y.drop(de_result.id)
    del cas_source[de_result.id]
    print(learn_x)
f.close()
		#結果の出力
result.to_csv("result_cand_1_EI.csv", index=False, header=False)



0   -6.783594
dtype: float64
141-76-4
-4.0
726
     0   1   2    3   4   5   6   7         8   9  ...  41  42  43  44  45  \
461   2   0   2  320  15  16  12   1  213.5800   0 ...   6   0   1   1   4   
21    1   0   0  225  11  11   4   2  142.0910   0 ...   0   0   0   3   3   
476   3   3   0  294  16  18   2   3  200.1520   0 ...   0   0   0   9   2   
889   0   0   0   97   4   3   2   1   71.4851   0 ...   0   0   0   3   1   
968   1   0   1  323  16  16  10   5  206.1370   0 ...   0   0   0   0   4   
409   1   0   1  286  14  14   8   4  197.5770   0 ...   0   0   0   3   3   
758   3   0   3  240  15  17  14   0  180.1600   0 ...   0   0   2   1   0   
23    1   0   1  105   6   6   6   0   74.0603   0 ...   0   0   0   0   1   
829   1   0   1  209  10  10   8   1  130.0810   0 ...   1   1   0   0   2   
532   1   0   0  334  16  16   6   4  224.1950   0 ...   0   0   0   8   2   
294   1   0   1  145   7   7   6   0   91.0622   0 ...   0   0   0   0   1   

     46  47  48 

In [76]:
import numpy as np
import pandas as pd
import sys
from PEIOPT_for_loop import *
import csv
import time

x_all = pd.read_csv("./../x_all.csv", header=None)
y_all = pd.read_csv("./../y_all.csv", header=None)
cas_data = pd.read_csv("./../sum_log.csv")
counter_i = 1

csv_title_list = ["trial","cas_number","pred_value","real_value","error_value"]

f = open('PEIOPT_process_10_1.csv', 'w')
writer = csv.writer(f, lineterminator='\n')
writer.writerow(csv_title_list)

sampler = np.random.permutation(len(x_all))

learn_x = x_all.take(sampler[:10])
learn_y = y_all.take(sampler[:10])
cas_source = (cas_data.ix[:,1]).take(sampler[10:])
cas_source = list(cas_source.values.flatten())
#
test_x = x_all.take(sampler[10:])
test_y = y_all.take(sampler[10:])
#

In [78]:
len(cas_source)

1144

In [80]:
test_x.iloc[1:]

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
908,0,0,0,163,4,3,0,0,162.8210,0,...,0,0,0,3,2,0,0,0,0,4
552,1,0,0,112,7,7,0,0,84.0749,0,...,0,0,0,7,0,0,0,0,0,1
922,0,0,0,137,8,7,0,5,98.0817,0,...,0,0,0,6,0,1,1,0,0,8
801,0,0,0,166,8,7,2,2,104.0660,0,...,0,0,0,0,3,0,0,0,0,8
1083,1,0,0,128,8,8,0,0,96.0856,0,...,0,0,0,8,0,0,0,0,0,1
744,2,0,2,339,15,16,12,1,250.4870,0,...,0,0,1,3,3,0,0,0,2,5
841,0,0,0,116,6,5,0,2,76.0529,0,...,0,0,0,3,1,0,0,0,0,6
849,0,0,0,132,7,6,2,3,88.0636,0,...,0,0,0,5,1,0,0,0,0,7
988,0,0,0,180,10,9,2,6,124.0960,0,...,0,0,0,8,1,0,0,0,0,10
957,2,0,0,516,20,21,12,1,280.1080,0,...,0,0,0,0,8,6,0,0,0,10


In [46]:
test_x

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
839,0,0,0,116,6,5,0,2,76.0529,0,...,0,0,0,4,1,1,0,0,0,6
51,1,1,0,116,6,6,0,0,76.0529,0,...,0,0,0,3,1,0,0,0,0,2
35,0,0,0,120,5,4,0,2,68.0309,0,...,0,0,0,0,2,0,0,0,0,5
463,1,0,1,279,13,13,8,3,187.5620,0,...,0,0,0,1,3,1,0,0,1,8
327,0,0,0,113,5,4,0,2,83.4958,0,...,0,0,0,4,1,0,0,0,0,5
898,0,0,0,166,8,7,2,4,120.1320,0,...,0,0,0,2,0,2,0,0,0,8
526,1,0,1,233,10,10,8,1,151.5270,0,...,0,1,0,1,3,0,0,1,1,5
132,3,0,1,741,32,34,9,7,430.2140,0,...,0,0,0,0,12,7,0,0,1,17
782,5,1,4,320,20,24,20,0,240.2140,0,...,0,0,8,0,0,0,0,0,4,1
267,3,0,0,196,11,12,0,0,136.1060,0,...,0,0,0,8,1,0,0,0,0,4


In [75]:
cas_source

['539-03-7',
 '110-94-1',
 '583-60-8',
 '91-01-0',
 '628-63-7',
 '591-78-6',
 '2974-92-7',
 '29973-13-5',
 '434-03-7',
 '89-83-8',
 '121-98-2',
 '1689-84-5',
 '142-82-5',
 '95-76-1',
 '622-50-4',
 '1678-91-7',
 '598-53-8',
 '38380-08-4',
 '119-91-5',
 '108-87-2',
 '107-83-5',
 '55-21-0',
 '1634-04-4',
 '110-82-7',
 '3248-05-3',
 '142-28-9',
 '437-50-3',
 '1540-35-8',
 '58-15-1',
 '57-83-0',
 '485-71-2',
 '38444-86-9',
 '77-27-0',
 '108-39-4',
 '77-92-9',
 '100-47-0',
 '75-08-1',
 '467-98-1',
 '5221-53-4',
 '127-07-1',
 '52663-62-4',
 '5989-27-5',
 '598-50-5',
 '589-90-2',
 '539-82-2',
 '109-92-2',
 '50-03-3',
 '56-23-5',
 '93-65-2',
 '781-43-1',
 '120-12-7',
 '111-26-2',
 '496-11-7',
 '75-30-9',
 '108-43-0',
 '111-76-2',
 '88-67-5',
 '41464-41-9',
 '1698-60-8',
 '3766-60-7',
 '88-99-3',
 '13993-65-2',
 '98-06-6',
 '99-87-6',
 '108-21-4',
 '3970-62-5',
 '15879-93-3',
 '565-60-6',
 '595-41-5',
 '53-19-0',
 '60-87-7',
 '5635-50-7',
 '260-94-6',
 '93-89-0',
 '458-88-8',
 '86-74-8',
 '6988-

In [61]:
test_x

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
466,1,0,1,196,11,11,8,3,136.1060,0,...,0,0,0,3,1,0,0,0,1,6
654,1,0,0,128,8,8,0,0,96.0856,0,...,0,0,0,8,0,0,0,0,0,3
150,2,0,1,460,20,21,6,4,268.1350,0,...,0,0,0,0,7,5,0,0,1,10
152,2,0,2,245,11,12,10,0,163.5440,0,...,0,0,0,1,5,0,0,0,2,2
954,1,0,0,196,11,11,4,1,136.1060,0,...,0,0,0,9,1,0,0,0,0,6
745,3,0,3,224,14,16,14,0,168.1500,0,...,0,0,2,0,0,0,0,0,3,1
1067,3,0,3,448,23,25,18,4,292.2010,0,...,0,0,6,3,4,1,0,0,3,9
1062,0,0,0,112,7,6,2,3,84.0749,0,...,0,0,0,7,0,0,0,0,0,7
525,1,0,1,318,12,12,8,1,385.8810,0,...,0,1,0,2,3,1,0,1,1,7
849,0,0,0,132,7,6,2,3,88.0636,0,...,0,0,0,5,1,0,0,0,0,7


In [33]:
learn_x.append(test_x.ix[de_result.id,:])

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
16,1,0,0,141,7,7,0,0,90.0597,0,...,0,0,0,0,2,0,0,0,0,2
667,3,1,3,522,25,27,18,5,341.6610,0,...,0,11,5,3,6,0,0,1,3,12
1003,3,0,2,368,18,20,16,0,232.1470,0,...,0,0,2,0,4,2,0,0,2,5
13,1,0,1,141,7,7,6,0,90.0597,0,...,0,0,0,0,2,1,0,0,1,2
1019,1,0,1,238,11,11,8,2,205.9960,0,...,0,0,0,2,1,1,0,0,1,6
709,3,1,2,224,14,16,12,0,168.1500,0,...,0,0,1,2,0,0,0,0,2,2
883,0,0,0,148,8,7,2,4,100.0740,0,...,0,0,0,6,1,0,0,0,0,8
1131,2,0,2,388,16,17,12,1,285.9400,0,...,0,0,1,4,4,0,0,0,2,6
53,0,0,0,116,6,5,0,1,76.0529,0,...,0,0,0,4,1,1,0,0,0,6
117,1,0,1,148,8,8,6,0,100.0740,0,...,0,0,0,1,1,1,0,0,1,3


In [34]:
learn_x

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
16,1,0,0,141,7,7,0,0,90.0597,0,...,0,0,0,0,2,0,0,0,0,2
667,3,1,3,522,25,27,18,5,341.6610,0,...,0,11,5,3,6,0,0,1,3,12
1003,3,0,2,368,18,20,16,0,232.1470,0,...,0,0,2,0,4,2,0,0,2,5
13,1,0,1,141,7,7,6,0,90.0597,0,...,0,0,0,0,2,1,0,0,1,2
1019,1,0,1,238,11,11,8,2,205.9960,0,...,0,0,0,2,1,1,0,0,1,6
709,3,1,2,224,14,16,12,0,168.1500,0,...,0,0,1,2,0,0,0,0,2,2
883,0,0,0,148,8,7,2,4,100.0740,0,...,0,0,0,6,1,0,0,0,0,8
1131,2,0,2,388,16,17,12,1,285.9400,0,...,0,0,1,4,4,0,0,0,2,6
53,0,0,0,116,6,5,0,1,76.0529,0,...,0,0,0,4,1,1,0,0,0,6
117,1,0,1,148,8,8,6,0,100.0740,0,...,0,0,0,1,1,1,0,0,1,3


In [32]:
test_x.ix[de_result.id,:]

0       1.000
1       0.000
2       0.000
3     318.000
4      15.000
5      15.000
6       6.000
7       4.000
8     196.119
9       0.000
10      0.000
11      5.000
12      4.000
13      0.000
14      0.000
15      0.000
16      0.000
17      3.000
18      0.000
19      0.000
20      0.000
21      0.000
22     75.270
23     56.000
24     95.000
25     54.000
26      0.000
27      0.000
28     12.000
29     16.000
30      0.000
31      0.000
32      2.000
33      0.000
34      0.000
35      0.000
36      0.000
37      0.000
38      0.000
39      0.000
40      0.000
41      0.000
42      0.000
43      0.000
44      7.000
45      3.000
46      2.000
47      0.000
48      0.000
49      0.000
50     10.000
Name: 268, dtype: float64

In [22]:
cas_source

['853-23-6',
 '88-96-0',
 '58-40-2',
 '52645-53-1',
 '57-43-2',
 '95-57-8',
 '539-03-7',
 '57-85-2',
 '4248-20-8',
 '111-71-7',
 '51953-17-4',
 '78-83-1',
 '98-54-4',
 '598-75-4',
 '102170-44-5',
 '614-28-8',
 '88-99-3',
 '98-01-1',
 '620-71-3',
 '87-72-9',
 '51-79-6',
 '111-11-5',
 '110-54-3',
 '60168-88-9',
 '1185-33-7',
 '98-85-1',
 '789-02-6',
 '564-25-0',
 '142-68-7',
 '74-88-4',
 '110-42-9',
 '6893-02-3',
 '694-80-4',
 '615-77-0',
 '123-38-6',
 '474-25-9',
 '1912-26-1',
 '2050-67-1',
 '3813-05-6',
 '91-94-1',
 '111-82-0',
 '52-43-7',
 '74-95-3',
 '1540-35-8',
 '598-55-0',
 '146-54-3',
 '2303-16-4',
 '91-57-6',
 '128-62-1',
 '110-38-3',
 '600-36-2',
 '95-14-7',
 '61-68-7',
 '624-95-3',
 '38380-08-4',
 '583-58-4',
 '112-53-8',
 '565-60-6',
 '120-72-9',
 '126-07-8',
 '107-06-2',
 '2051-24-3',
 '31367-46-1',
 '52-31-3',
 '107-05-1',
 '230-46-6',
 '33284-53-6',
 '1125-78-6',
 '629-14-1',
 '507-45-9',
 '89-83-8',
 '127-07-1',
 '1732-13-4',
 '129-00-0',
 '2008-41-5',
 '50-11-3',
 '94-96

0       3.000
1       0.000
2       2.000
3     457.000
4      22.000
5      24.000
6      12.000
7       3.000
8     286.182
9       0.000
10      0.000
11      8.000
12      0.000
13      0.000
14      0.000
15      0.000
16      1.000
17      3.000
18      1.000
19      0.000
20      1.000
21      1.000
22    100.350
23      0.000
24     36.000
25      9.000
26      0.000
27      5.000
28     92.000
29     51.000
30      0.000
31     25.000
32      0.000
33      0.000
34     10.000
35      0.000
36      0.000
37      0.000
38      5.000
39     23.000
40     10.000
41      0.000
42      4.000
43      0.000
44      1.000
45      7.000
46      1.000
47      0.000
48      1.000
49      2.000
50      8.000
Name: 1013, dtype: float64